In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
blocks = 32
batches = 4
max_iters = 10000
learning_rate = 1e-3
training_percent = 0.8
eval_iters = 500

In [2]:
with open('grammar.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_len = len(chars)

['\n', ' ', '!', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '}', '´', 'Á', 'à', 'á', 'æ', 'è', 'é', 'í', 'ò', 'ó', 'ù', 'ú', 'Ā', 'ā', 'Ă', 'ă', 'ē', 'ĕ', 'ħ', 'Ī', 'ī', 'Ĭ', 'ĭ', 'ō', 'Ŏ', 'ŏ', 'œ', 'Ū', 'ū', 'ŭ', 'ȳ', '̄', '̆', 'Α', 'Κ', 'Λ', 'Ν', 'Ο', 'Τ', '‘', '’', '“', '”', '⪥', '\ufeff']


In [3]:
str2int = { ch:i for i,ch in enumerate(chars) }
int2str = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [ str2int[c] for c in s ]
decode = lambda l: ''.join([ int2str[i] for i in l ])

data = torch.tensor(encode(text), dtype=torch.long).to(device)
print(data[:100])

tensor([125,   0,   0,   0,   0,  39,  71,  68,  57,  74,  56,  58,  57,   1,
         55,  78,   1,  26,  61,  71,  62,  72,   1,  26,  74,  71,  67,  68,
         76,   1,  54,  67,  57,   1,  73,  61,  58,   1,  38,  67,  65,  62,
         67,  58,   1,  27,  62,  72,  73,  71,  62,  55,  74,  73,  58,  57,
          0,  39,  71,  68,  68,  59,  71,  58,  54,  57,  62,  67,  60,   1,
         43,  58,  54,  66,   1,  54,  73,   1,  61,  73,  73,  69,  21,  10,
         10,  76,  76,  76,   9,  69,  60,  57,  69,   9,  67,  58,  73,   1,
          4,  43])


In [4]:
n = int(training_percent*len(data))
train = data[:n]
valid = data[n:]

def get_batch(split):
    data = train if split=='train' else valid
    ix = torch.randint(len(data) - blocks, (batches,))
    # print(ix)
    x = torch.stack([data[i:i+blocks] for i in ix])
    y = torch.stack([data[i+1:i+blocks+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('target:')
print(y)

inputs:
tensor([[ 67,  54,  71,  78,   7,   1,  54,  67,  57,   1,  62,  59,   1,  78,
          68,  74,   0,  61,  54,  75,  58,   1,  67,  68,   1,  57,  62,  56,
          73,  62,  68,  67],
        [ 52,  43,  61,  58,  78,  52,   1,  73,  58,  65,  65,   1,  74,  72,
           7,   1,  76,  68,  66,  58,  67,   1,  56,  54,  67, 121,  73,   1,
          72,  54,  78,   1],
        [ 58,  71,  72,  23, 123,   1, 122,  27,  68,  67, 121,  73,   1,  78,
          68,  74,   1,  76,  62,  72,  61,   1,  78,  68,  74,   1,  76,  54,
          72,   1,  68,  67],
        [ 58,  52,   7,   1,  52,  54,  67,  52,   1,  61,  68,  74,  67,  57,
           7,   1,  52,  54,  67,  52,   0,  61,  74,  67,  73,  58,  71,   7,
           1,   3,  56,   9]])
target:
tensor([[ 54,  71,  78,   7,   1,  54,  67,  57,   1,  62,  59,   1,  78,  68,
          74,   0,  61,  54,  75,  58,   1,  67,  68,   1,  57,  62,  56,  73,
          62,  68,  67,  54],
        [ 43,  61,  58,  78,  52,   1,  73,

In [5]:
x = train[:blocks]
y = train[1:blocks+1]
for t in range(blocks):
    ctx = x[:t+1]
    target = y[t]
    print('When input is', ctx, 'target is', target)

In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
class Bigram(nn.Module):
    def __init__(self, vocab_len):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_len, vocab_len)
    def forward(self, index, targets=None):
        logits = self.token_embedding(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index
model = Bigram(vocab_len).to(device)

ctx = torch.zeros((1, 1), dtype=torch.long, device=device)
generated = decode(model.generate(ctx, max_new_tokens=500)[0].tolist())
print(generated)


lĕFS}ĬAùùr*a⪥QēFskYŎEfU,ĭ!Á7h6HÁe5ō̄zp.47Ο.&P&EmeR0
wIHelĬXM⪥47.í,ĕĪ)ΛŎl5FēāWv SJ`]ŪD’]iè
”2lōΑO:ZRŭptīΛ2sΤĀVīoBv út﻿W4ZF7b“IB.ĕJ(`
7.ĭ9vħEt̄opPyl,bs̄Ŏ/aQΤ̆)MΑOfŪqE8,ȳ“Zī0[H”]S
áL0ĭsV_/tK0bLMnt[Br?īzá?U’[UΤd´_ūΟ}Vh/YeòD63﻿X1eΝpÁ!̄ ‘H?á?BMCO_æ’ĭ̆:CΑ*yjEv/ŎΟȳ
cBĭùāΤ-*dZNĕŭaQ⪥āCx(ăvbF” ūæèsBΤyœIH)
LríáTXŏJq
ΚNq]7Ο!wH3W﻿òPāuR“IKX]pΤcu_AħSy)9`]BHmzáΛBtD/ĂŭΟMóŪT9﻿ o
[[0ĕvħōy6eèœΤĬ⪥]sRPΟPQăòsR0mó1ūo5NsùvēlneΑ1ŭ⪥﻿i/K,óòwI:⪥sΤĀá4⪥áLO_Ĭá﻿(nōbī̄UΝm9ΤNĕjĀŎ̆_œ4/ma9’K4!ī!n6j]o}āĪGS̆4Ĭa.ĭPf&]Ă̄íJU”8*jSŏQòOsm_Ĭ


In [8]:
%%time
optim = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.6f}, valid loss: {losses['val']:.6f}")
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optim.zero_grad(set_to_none=True)
    loss.backward()
    optim.step()
print(loss.item())

step: 0, train loss: 5.357635, valid loss: 5.354078
step: 500, train loss: 4.813578, valid loss: 4.816730
step: 1000, train loss: 4.349066, valid loss: 4.355768
step: 1500, train loss: 3.968702, valid loss: 3.968378
step: 2000, train loss: 3.658509, valid loss: 3.668593
step: 2500, train loss: 3.405668, valid loss: 3.420797
step: 3000, train loss: 3.229121, valid loss: 3.241028
step: 3500, train loss: 3.069561, valid loss: 3.108018
step: 4000, train loss: 2.962914, valid loss: 3.012167
step: 4500, train loss: 2.869317, valid loss: 2.924562
step: 5000, train loss: 2.813273, valid loss: 2.858808
step: 5500, train loss: 2.764403, valid loss: 2.815550
step: 6000, train loss: 2.714870, valid loss: 2.776400
step: 6500, train loss: 2.687440, valid loss: 2.753028
step: 7000, train loss: 2.665990, valid loss: 2.734124
step: 7500, train loss: 2.646038, valid loss: 2.714727
step: 8000, train loss: 2.631454, valid loss: 2.712579
step: 8500, train loss: 2.607819, valid loss: 2.704341
step: 9000, tr

In [9]:
ctx = torch.zeros((1, 1), dtype=torch.long, device=device)
generated = decode(model.generate(ctx, max_new_tokens=500)[0].tolist())
print(generated)


ct  lles ay  f tigon  s! hoshe ay htutinengl ppefe  isoriooroursŏbud. XÁ﻿JEDre﻿ū8qQP. thenllltora  Andhayst n_.
  wo inthavelsa
“ICAc Ac..
Th.
 lie
  Iòū9be Cg  s pl e a
 d 1̄S MKòw aisí0Ŏcto paby CTh, r Joogin, ar/ sthowafofe msonaspe_ave iby, asugg, th l`A be’lle8  chlesespassythe t /àāūy ditis duthape, oumblilu’F lise iahoa   PPECo f rder if nveanarins. bofuuindithanP9ZOOn,”]Ā_ardins the   t bered ix  thaders  agar” wa harers,  wininseyenlchemsherealy ays whtrncayonake  qun’ĂPHed. CHe d th,-a


In [11]:
import speech_recognition as sr
import pyttsx3


listener = sr.Recognizer()
engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

def talk(audio):
    engine.say(audio)
    engine.runAndWait()

# talk(generated)